In [1]:
# curva_intermedia_unitec = spark.read.parquet('abfss://cecbe0eb-13fe-49f5-9c97-8673c649bc01@onelake.dfs.fabric.microsoft.com/1047c72d-ff7e-4727-91ce-5f96efe88de5/Files/ANALYTIKUS/CURVA_INTERMEDIA_UNITEC.parquet')

StatementMeta(, f7c71941-bc1a-4cf1-b886-8482f2c4a9ca, 3, Finished, Available, Finished)

#### Importación de módulos

In [2]:
import pyspark.sql.functions as f_
from pyspark.sql.functions import current_timestamp
from itertools import chain
from pyspark.sql.window import Window
import sys
sys.path.append('/lakehouse/default/Files/GENERALES')
#%run "/lakehouse/default/Files/GENERALES/f_amr.py"
import f_amr

StatementMeta(, f7c71941-bc1a-4cf1-b886-8482f2c4a9ca, 4, Finished, Available, Finished)

#### Ajuste a curva de intentos

In [3]:
curva_bdd_unitec = spark.read.parquet('abfss://8c61db5f-46e1-4c72-8d6f-648bf05cacac@onelake.dfs.fabric.microsoft.com/0d2e9829-8a43-4592-a9d7-db37f7032a1c/Files/BASES_EN_PROCESO/CURVA_INTENTOS_UNITEC')
curva_bdd_unitec = curva_bdd_unitec.withColumn('INDICE', f_.row_number().over(Window.orderBy('NO_DE_PERSONA', 'SEMANA')))
registros_marcados = curva_bdd_unitec.select('NO_DE_PERSONA').distinct()

StatementMeta(, f7c71941-bc1a-4cf1-b886-8482f2c4a9ca, 5, Finished, Available, Finished)

- ##### Adición de segmentos de marcación

In [4]:
# Preparación de Segmentos P1
segmentos = spark.read.parquet('abfss://cecbe0eb-13fe-49f5-9c97-8673c649bc01@onelake.dfs.fabric.microsoft.com/1047c72d-ff7e-4727-91ce-5f96efe88de5/Files/ANALYTIKUS/SEGMENTOS.parquet')
segmentos = segmentos.drop('CARRERA', 'ETAPA', 'INSCRITOS', 'Id_Persona_Ciclo')
segmentos = segmentos.withColumn('FECHA', f_.to_date('FECHA', "yyyy-MM-dd"))
segmentos = f_amr.cast_cols(segmentos, {'SEMANA':'int'})
#segmentos = segmentos.withColumnRenamed('TIPO DE PROGRAMA', 'TIPO_DE_PROGRAMA').withColumnRenamed('AGRUPADOR SITIO', 'AGRUPADOR_SITIO')

match_segmentos = segmentos.join(registros_marcados,  'NO_DE_PERSONA', 'inner')
match_segmentos = match_segmentos.withColumn('ESTATUS_MARCACION', f_.lit('MARCADO'))

no_match_segmentos_left = segmentos.join(registros_marcados,  'NO_DE_PERSONA', 'left_anti')
no_match_segmentos_left = no_match_segmentos_left.withColumn('ESTATUS_MARCACION', f_.lit('NO MARCADO'))
no_match_segmentos_left = f_amr.unique_records(no_match_segmentos_left, ['NO_DE_PERSONA', 'SEMANA'], [('NO_DE_PERSONA', 'asc'),('SEMANA', 'asc')], 'REG_UNICOS_SEM')

segmentos = match_segmentos.unionByName(no_match_segmentos_left, allowMissingColumns=True)

# Adición de segmentos
match_curva_seg = segmentos.select('APELLIDO_MATERNO', 'APELLIDO_PATERNO', 'NOMBRE', 'NOMBRE_COMPLETO', 'SEGMENTO', 'NO_DE_PERSONA', 'SEMANA').join(curva_bdd_unitec, ['NO_DE_PERSONA', 'SEMANA'], 'inner').dropDuplicates(['INDICE'])
no_match_curva_seg_left = segmentos.join(curva_bdd_unitec, ['NO_DE_PERSONA', 'SEMANA'], 'left_anti')
no_match_curva_seg_right = curva_bdd_unitec.join(segmentos, ['NO_DE_PERSONA', 'SEMANA'], 'left_anti')

curva_segmentos = match_curva_seg.unionByName(no_match_curva_seg_left, allowMissingColumns=True).unionByName(no_match_curva_seg_right, allowMissingColumns=True)

# Normalización post uniones
curva_segmentos = curva_segmentos.withColumn('SEGMENTO', f_.when(f_.col('SEGMENTO').isNull(), 'OTROS').otherwise(f_.col('SEGMENTO')))
curva_segmentos = curva_segmentos.withColumnRenamed('SEGMENTO', 'SEGMENTO_VIGENTE').drop('INDICE')
curva_segmentos = curva_segmentos.withColumn('INDICE', f_.row_number().over(Window.orderBy('NO_DE_PERSONA', 'SEMANA')))

curva_segmentos_otros = curva_segmentos.filter(f_.col('SEGMENTO_VIGENTE') == 'OTROS' )

curva_segmentos_vigentes = curva_segmentos.filter(f_.col('SEGMENTO_VIGENTE') != 'OTROS' )

# Preparación de segmentos P2
segmentos_sin_insercion = spark.read.parquet('abfss://cecbe0eb-13fe-49f5-9c97-8673c649bc01@onelake.dfs.fabric.microsoft.com/1047c72d-ff7e-4727-91ce-5f96efe88de5/Files/ANALYTIKUS/SEGMENTOS_SIN_INSERCION.parquet')
segmentos_sin_insercion = segmentos_sin_insercion.withColumn('FECHA', f_.to_date('FECHA', "yyyy-MM-dd"))
segmentos_sin_insercion = f_amr.cast_cols(segmentos_sin_insercion, {'SEMANA':'int'})
#segmentos_sin_insercion = segmentos_sin_insercion.withColumnRenamed('TIPO DE PROGRAMA', 'TIPO_DE_PROGRAMA').withColumnRenamed('AGRUPADOR SITIO', 'AGRUPADOR_SITIO')
segmentos_sin_insercion = segmentos_sin_insercion.drop('CARRERA', 'ETAPA', 'INSCRITOS', 'Id_Persona_Ciclo')
segmentos_sin_insercion = segmentos_sin_insercion.withColumn('SEGMENTO', f_.concat_ws( '', 'SEGMENTO', f_.lit(' OTRAS SEMANAS')))
segmentos_sin_insercion = f_amr.uniques(segmentos_sin_insercion,['NO_DE_PERSONA'],[('NO_DE_PERSONA', 'asc'),('FECHA', 'desc')]) #filter('INDICE = 1')

# Adición de segmentos
match_curva_segmentos_otros = curva_segmentos_otros.join(segmentos_sin_insercion.select('NO_DE_PERSONA', 'SEGMENTO'), 'NO_DE_PERSONA', 'inner').dropDuplicates(['INDICE'])
no_match_curva_segmentos_otros_left = curva_segmentos_otros.join(segmentos_sin_insercion, 'NO_DE_PERSONA', 'left_anti')
curva_segmentos = match_curva_segmentos_otros.unionByName(no_match_curva_segmentos_otros_left, allowMissingColumns=True).unionByName(curva_segmentos_vigentes, allowMissingColumns=True)

# Normalización post uniones
curva_segmentos = curva_segmentos.withColumn('SEGMENTO', f_.when(f_.col('SEGMENTO').isNull(), 'OTROS').otherwise(f_.col('SEGMENTO')))
curva_segmentos = curva_segmentos.withColumn('SEGMENTO1', f_.col('SEGMENTO_VIGENTE'))
curva_segmentos = curva_segmentos.withColumn('SEGMENTO1', f_.when(f_.col('SEGMENTO1') == 'OTROS', f_.col('SEGMENTO')).otherwise(f_.col('SEGMENTO1')))
curva_segmentos = curva_segmentos.drop('SEGMENTO', 'INDICE').withColumnRenamed('SEGMENTO1', 'SEGMENTO_DESGLOSE')
curva_segmentos = curva_segmentos.withColumn('SEGMENTO_GENERAL', f_.when(f_.col('SEGMENTO_DESGLOSE').contains("ABS"), "ABSORCION")
                                                                   .when(f_.col("SEGMENTO_DESGLOSE").contains("EXA"), "EXALUMNOS")
                                                                   .when(f_.col("SEGMENTO_DESGLOSE").contains("CSA"), "CSA")
                                                                   .when(f_.col("SEGMENTO_DESGLOSE").contains("ANC"), "ANC")
                                                                   .otherwise(f_.col("SEGMENTO_DESGLOSE")))
curva_segmentos = curva_segmentos.withColumn('INDICE', f_.row_number().over(Window.orderBy('NO_DE_PERSONA', 'SEMANA')))

# Preparación de segmentos P3
segmentos_ccenter = spark.read.parquet('abfss://cecbe0eb-13fe-49f5-9c97-8673c649bc01@onelake.dfs.fabric.microsoft.com/1047c72d-ff7e-4727-91ce-5f96efe88de5/Files/ANALYTIKUS/SEGMENTOS_CCENTER.parquet')
segmentos_ccenter = f_amr.cast_cols(segmentos_ccenter, {'SEMANA':'int'})
segmentos_ccenter = segmentos_ccenter.drop('FECHA_ACTUAL')
segmentos_ccenter = segmentos_ccenter.withColumnRenamed('SEGMENTO', 'SEGMENTO_CCENTER').withColumnRenamed('SEGMENTO_AGRUPADO', 'SEGMENTO_AGRUPADO_CCENTER').withColumnRenamed('NO_PERSONA', 'NO_DE_PERSONA')
segmentos_ccenter = segmentos_ccenter.filter(~f_.col('SEGMENTO_AGRUPADO_CCENTER').contains('POT'))

# Adición de segmentos
match_curva_segmentos = curva_segmentos.join(segmentos_ccenter, ['NO_DE_PERSONA', 'SEMANA'], 'inner').dropDuplicates(['INDICE'])
no_match_curva_segmentos_left = curva_segmentos.join(segmentos_ccenter, ['NO_DE_PERSONA', 'SEMANA'], 'left_anti')
curva_segmentos = match_curva_segmentos.unionByName(no_match_curva_segmentos_left, allowMissingColumns=True)
curva_segmentos = curva_segmentos.drop('INDICE')
curva_segmentos = curva_segmentos.withColumn('INDICE', f_.row_number().over(Window.orderBy('NO_DE_PERSONA', 'SEMANA')))

StatementMeta(, f7c71941-bc1a-4cf1-b886-8482f2c4a9ca, 6, Finished, Available, Finished)

- ##### Adición de score

In [5]:
# Fuente
score_compilado = spark.read.parquet('abfss://cecbe0eb-13fe-49f5-9c97-8673c649bc01@onelake.dfs.fabric.microsoft.com/1047c72d-ff7e-4727-91ce-5f96efe88de5/Files/ANALYTIKUS/SCORE_COMPILADO.parquet')
score_compilado = f_amr.cast_cols(score_compilado, {'CALL_SCORE':'int','SCORE_UNICOS':'int','LEAD_SCORE':'int'})
score_compilado = score_compilado.withColumnRenamed('ID_Persona','NO_DE_PERSONA')

# Adición de score
match_segmentos_score_1= curva_segmentos.join(score_compilado, ['NO_DE_PERSONA', 'CICLO'], 'inner').dropDuplicates(['INDICE'])
no_match_segmentos_score_left_1= curva_segmentos.join(score_compilado, ['NO_DE_PERSONA', 'CICLO'], 'left_anti')
no_match_segmentos_score_right_1 = score_compilado.join(curva_segmentos, ['NO_DE_PERSONA', 'CICLO'], 'left_anti')
match_segmentos_score_2 = no_match_segmentos_score_left_1.join(no_match_segmentos_score_right_1.drop('CICLO'), 'NO_DE_PERSONA', 'inner').dropDuplicates(['INDICE'])
no_match_segmentos_score_left_2 = no_match_segmentos_score_left_1.join(no_match_segmentos_score_right_1.drop('CICLO'), 'NO_DE_PERSONA', 'left_anti')

# Curva segmentos final
curva_segmentos = match_segmentos_score_1.unionByName(match_segmentos_score_2, allowMissingColumns=True).unionByName(no_match_segmentos_score_left_2, allowMissingColumns=True)
curva_segmentos = curva_segmentos.drop('INDICE').withColumn('BASE_MARCACION', f_.lit(1))



StatementMeta(, f7c71941-bc1a-4cf1-b886-8482f2c4a9ca, 7, Finished, Available, Finished)

In [6]:
from pyspark.sql.functions import current_timestamp
curva_segmentos = curva_segmentos.withColumn("ACTUALIZACION", f_.expr("current_timestamp() - INTERVAL 6 HOURS"))
#curva_segmentos.repartition(5).write.option("overwriteSchema", "true").format("delta").mode("overwrite").saveAsTable("CURVAS.CURVA_SEGMENTOS")
p1= 'abfss://8c61db5f-46e1-4c72-8d6f-648bf05cacac@onelake.dfs.fabric.microsoft.com/0d2e9829-8a43-4592-a9d7-db37f7032a1c/Files/BASES_EN_PROCESO/CURVA_SEGMENTOS'
curva_segmentos.repartition(10).write.mode("overwrite").parquet(p1)

StatementMeta(, f7c71941-bc1a-4cf1-b886-8482f2c4a9ca, 8, Finished, Available, Finished)